## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

Прочтем файл и выведем первые 5 строчек на экран

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


### Предобработка данных

Проверим данные на наличие дубликатов

In [3]:
data.duplicated().sum()

0

Явные дубликаты отсутствуют. Далее необходимо проверить данные на наличие пропусков

In [4]:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Пропуски присутствуют в столбце **Tenure**, который показывает, сколько лет человек является клиентом банка. Заполнение подобных пропусков синтетическими значениями (например, медианой) может сильно испортить исходное распределение. Но в то же время библиотеки машинного обучения не умеют работать с пропусками. Поэтому остается одно - избавиться от строк с пропусками в данном столбце.

In [5]:
data.dropna(inplace=True)

### Разделение выборок

Чтобы преобразовать категориальные признаки в количественные, применим прямое кодирование ко всему датафрейму. От столбцов **Surname** и **RowNumber** можно избавиться, тк они мало чем помогут построению моделей, а объем данных значительно возрастет, если эти столбцы оставить

In [6]:
data_ohe = pd.get_dummies(data.drop(['Surname', 'RowNumber'], axis=1))

Разделим данные на признаки и целевой признак

In [7]:
features = data_ohe.drop('Exited', axis=1)
target = data_ohe['Exited']

Разделим данные на обучающую, валидационную и тестовую выборки

In [8]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
                                        features, target, test_size=0.4, random_state=42)
features_valid, features_test, target_valid, target_test = train_test_split(
                    features_valid_test, target_valid_test, test_size=0.5, random_state=42)

In [9]:
print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)

(5454, 14) (1818, 14) (1819, 14)
(5454,) (1818,) (1819,)


В итоге имеем:
- features_train, target_train - обучающая выборка (размеры (5454, 14) и (5454,) соответственно)
- features_valid, target_valid - валидационная выборка (размеры (1818, 14) и (1818,) соответственно)
- features_test, target_test - тестовая выборка (размеры (1819, 14) и (1819,) соответственно)

## Исследование задачи

### Исследование баланса классов

Посмотрим, сколько положительных и отрицательных объектов в обучающей выборке 

In [10]:
target_train.value_counts()

0    4343
1    1111
Name: Exited, dtype: int64

Количество отрицательных объектов: 4343, количество положительных объектов: 1111

Наблюдается явный дисбаланс классов

### Исследование моделей без учета дисбаланса классов

#### Решающее дерево

Рассмотрим решающее дерево с глубиной от 1 до 15 и найдем наилучшие значения гиперпараметров

In [11]:
best_depth = 1
best_f1 = 0
for depth in range(1, 16):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    if best_f1 < f1_score(target_valid, predict_valid):
        best_depth = depth
        best_f1 = f1_score(target_valid, predict_valid)
print(best_depth)
print(best_f1)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


8
0.5209003215434084


Измерим значение AUC-ROC

In [12]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.677207363693317

**Вывод:** наибольшее значение f1 = 0.52 достигается при глубине дерева 8. Значение AUC-ROC = 0.67

#### Случайный лес

Рассмотрим случайный лес с количеством деревьев от 1 до 200 и найдем то количество, при котором достигается наибольшее значение f1 метрики

In [13]:
%%time
best_estimators = 1
best_f1 = 0
for estimators in range(1, 201):
    model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=42)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    f1_score(target_valid, predict_valid)
    if best_f1 < f1_score(target_valid, predict_valid):
        best_estimators = estimators
        best_f1 = f1_score(target_valid, predict_valid)

print(best_estimators)
print(best_f1)

59
0.5145797598627787
CPU times: user 2min 3s, sys: 530 ms, total: 2min 4s
Wall time: 2min 4s


Измерим значение AUC-ROC

In [14]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8536649515887758

**Вывод:** наибольшее значение f1 = 0.51 достигается при количестве дереьвев 59. Значение AUC-ROC = 0.853

#### Логистическая регрессия

Посмотрим, какое качество обеспечит логистическая регрессия

In [15]:
model = LogisticRegression(random_state=42)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
f1_score(target_valid, predict_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

Измерим значение AUC-ROC

In [16]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.6047065098073596

Теперь выставим параметр class_weight='balanced' и посмотрим, как поменяется значение f1

In [17]:
model = LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
f1_score(target_valid, predict_valid)

0.4919278252611586

Измерим значение AUC-ROC

In [18]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.7458725801237267

**Вывод:** логистическая регрессия обеспечивает низкое качество метрики f1 = 0 и значение AUC-ROC = 0.604, но при параметре class_weight='balanced' значение f1 поднимается до 0.49 и значение AUC-ROC = 0.74

## Борьба с дисбалансом

Рассмотрим два метода борьбы с дисбалансом: увеличение выборки (upsampling) и уменьшение выборки (downsampling)

### Upsampling

Увеличим выборку положительного класса, чтобы устранить дисбаланс

In [19]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones]*repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones]*repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

#### Решающее дерево

Рассмотрим решающее дерево с глубиной от 1 до 15 и найдем наилучшие значения гиперпараметров

In [20]:
best_depth = 1
best_f1 = 0
for depth in range(1, 16):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(features_upsampled, target_upsampled)
    predict_valid = model.predict(features_valid)
    if best_f1 < f1_score(target_valid, predict_valid):
        best_depth = depth
        best_f1 = f1_score(target_valid, predict_valid)
print(best_depth)
print(best_f1)

4
0.5702127659574469


Измерим значение AUC-ROC

In [21]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.6789288343132474

**Вывод:** наибольшее значение f1 = 0.57 достигается при глубине дерева 4. Значение AUC-ROC = 0.678

#### Случайный лес

Рассмотрим случайный лес с количеством деревьев от 1 до 200 и глубиной дерева от 1 до 15. Также, найдем значения гиперпараметров, при которых достигается наибольшее значение f1 метрики

In [40]:
%%time
best_estimators = 1
best_depth = 1
best_f1 = 0
for estimators in range(1, 201):
    for depth in range(1, 16):
        model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
        model.fit(features_upsampled, target_upsampled)
        predict_valid = model.predict(features_valid)
        f1_score(target_valid, predict_valid)
        if best_f1 < f1_score(target_valid, predict_valid):
            best_estimators = estimators
            best_depth = depth
            best_f1 = f1_score(target_valid, predict_valid)

print('количество деревьев:',best_estimators)
print('глубина:',best_depth)
print('f1:',best_f1)

количество деревьев: 163
глубина: 8
f1: 0.6295857988165681
CPU times: user 31min 6s, sys: 4.95 s, total: 31min 11s
Wall time: 31min 11s


Измерим значение AUC-ROC

In [41]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8520010155306786

**Вывод:** наибольшее значение f1 = 0.629 достигается при количестве дереьвев 163 и глубине 8. Значение AUC-ROC = 0.85

#### Логистическая регрессия

Посмотрим, какое качество обеспечит логистическая регрессия

In [42]:
model = LogisticRegression(random_state=42)
model.fit(features_upsampled, target_upsampled)
predict_valid = model.predict(features_valid)
f1_score(target_valid, predict_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.3828016643550624

Измерим значение AUC-ROC

In [43]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.6038590165790862

Теперь выставим параметр class_weight='balanced' и посмотрим, как поменяется значение f1

In [44]:
model = LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predict_valid = model.predict(features_valid)
f1_score(target_valid, predict_valid)

0.4919278252611586

Измерим значение AUC-ROC

In [45]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.745881712593859

**Вывод:** логистическая регрессия обеспечивает низкое качество метрики f1 = 0.38 и значение AUC-ROC = 0.6, но при параметре class_weight='balanced' значение f1 поднимается до 0.49 и значение AUC-ROC = 0.745

### Downsampling

Уменьшим выборку отрицательного класса, чтобы устранить дисбаланс

In [46]:
def downsample(features, target):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=0.25, random_state=42)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=0.25, random_state=42)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled
features_downsampled, target_downsampled = downsample(features_train, target_train)

#### Решающее дерево

Рассмотрим решающее дерево с глубиной от 1 до 15 и найдем наилучшие значения гиперпараметров

In [47]:
best_depth = 1
best_f1 = 0
for depth in range(1, 16):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(features_downsampled, target_downsampled)
    predict_valid = model.predict(features_valid)
    if best_f1 < f1_score(target_valid, predict_valid):
        best_depth = depth
        best_f1 = f1_score(target_valid, predict_valid)
print(best_depth)
print(best_f1)

6
0.5393939393939393


Измерим значение AUC-ROC

In [48]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.6737470707602051

**Вывод:** наибольшее значение f1 = 0.54 достигается при глубине дерева 6. Значение AUC-ROC = 0.67

#### Случайный лес

Рассмотрим случайный лес с количеством деревьев от 1 до 200 и глубиной дерева от 1 до 15. Также, найдем значения гиперпараметров, при которых достигается наибольшее значение f1 метрики

In [49]:
%%time
best_estimators = 1
best_depth = 1
best_f1 = 0
for estimators in range(1, 201):
    for depth in range(1, 16):
        model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
        model.fit(features_downsampled, target_downsampled)
        predict_valid = model.predict(features_valid)
        f1_score(target_valid, predict_valid)
        if best_f1 < f1_score(target_valid, predict_valid):
            best_estimators = estimators
            best_depth = depth
            best_f1 = f1_score(target_valid, predict_valid)

print('количество деревьев:',best_estimators)
print('глубина:',best_depth)
print('f1:',best_f1)

количество деревьев: 101
глубина: 8
f1: 0.6040554962646745
CPU times: user 12min 33s, sys: 3.44 s, total: 12min 36s
Wall time: 12min 37s


Измерим значение AUC-ROC

In [52]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8488283954067328

**Вывод:** наибольшее значение f1 = 0.604 достигается при количестве дереьвев 101 и глубине 8. Значение AUC-ROC = 0.85

#### Логистическая регрессия

Посмотрим, какое качество обеспечит логистическая регрессия

In [53]:
model = LogisticRegression(random_state=42)
model.fit(features_downsampled, target_downsampled)
predict_valid = model.predict(features_valid)
f1_score(target_valid, predict_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.38333333333333336

Измерим значение AUC-ROC

In [54]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.6031886932713787

Теперь выставим параметр class_weight='balanced' и посмотрим, как поменяется значение f1

In [55]:
model = LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predict_valid = model.predict(features_valid)
f1_score(target_valid, predict_valid)

0.4976258309591643

Измерим значение AUC-ROC

In [56]:
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
roc_auc_score(target_valid, probabilities_one_valid)

0.7460168731518164

**Вывод:** логистическая регрессия обеспечивает низкое качество метрики f1 = 0.38 и значение AUC-ROC = 0.60, но при параметре class_weight='balanced' значение f1 поднимается до 0.497 и значение AUC-ROC = 0.746

## Тестирование модели

Лучший результат на валидационной выборке показали модели случайного леса с количеством деревьев 163 и глубиной 8 при увеличении выборки положительного класса и с количеством деревьев 101 и глубиной 8 при уменьшении выборки отрицательного класса. Посмотрим, какой результат покажут эти модели на тестовой выборке

In [59]:
model = RandomForestClassifier(n_estimators=163, max_depth=8, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predict_test = model.predict(features_test)
f1_score(target_test, predict_test)

0.5904059040590405

In [60]:
model = RandomForestClassifier(n_estimators=101, max_depth=8, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predict_test = model.predict(features_test)
f1_score(target_test, predict_test)

0.567391304347826

**Вывод:** на тестовой выборке лучший результат f1 = 0.59 достигается при количестве деревьев 163 и глубине 8 на модели случайного леса при использовании upsampling в качестве метода борьбы с дисбалансом

## Общий вывод

В работе рассмотрено несколько моделей машинного обучения для решения задачи классификации: решающее дерево, случайный лес и логистическая регрессия. Исследован баланс классов и построены модели с учетом и без учета дисбаланса классов. Подобраны гиперпараметры моделей, при которых достигается наибольшее значение f1 метрики для каждой модели. Достичь лучшего результата на валидационной выборке (f1 = 0.6295) удалось с помощью случайного леса с количеством деревьев 163 и глубине 8 при использовании upsampling в качестве метода борьбы с дисбалансом. На тестовой выборке данная модель обеспечивает f1=0.59.